Problem Statemnet and its Deatils :
Manufacturing of any alloy is not a simple process. Many complicated factors are involved in the making of a perfect alloy, from the temperature at which various metals are melted to the presence of impurities to the cooling temperature set to cool down the alloy. Very minor changes in any of these factors can affect the quality or grade of the alloy produced.

Given are 28 distinguishing factors in the manufacturing of an alloy, your objective as a data scientist is to build a Machine Learning model that can predict the grade of the product using these factors.

You are provided with 28 anonymized factors (f0 to f27) that influence the making of a perfect alloy that is to be used for various applications based on the grade/quality of the obtained product.
Data Description

The unzipped folder will have the following files.

    Train.csv – 620 observations.
    Test.csv – 266 observations.
    Sample Submission – Sample format for the submission.

Target Variable: 

Socre = log loss

Note : This is version 2 program of xgboost which used for hackathon which help me to get in top 3% participation.
- Outlier handling is not done (need domain knowledge for this) you can do this by using various tech.

In [0]:
#mounting drive
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#chnage the current directory to working directory

import os 
os.chdir("/content/gdrive/My Drive/Hackathon/")

In [0]:
#unzip files
from zipfile import ZipFile

with ZipFile('MetalFurnace-Participants_Data.zip', 'r') as z:
  z.extractall()

In [0]:
#importing lib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV,StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
% matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
#lets read dataset and get deep drive into dataset

train_df = pd.read_csv("MetalFurnace-Participants_Data/Train.csv")
train_df.head(10)

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,grade
0,1.848564,-0.264250,-0.461423,0.409400,1.305455,2.329398,0.370965,0.090167,0.107958,0.0,0.395874,0.308879,0.548623,0.472101,0.172917,0.098853,0.308879,0.040193,0.182574,0.085505,0.233285,-1.080663,0.443257,-0.406121,-0.687687,0.271886,3.727218,0.102129,2
1,-0.825098,-0.264250,3.032397,-2.442599,1.305455,-0.276144,0.370965,0.090167,0.107958,0.0,0.395874,0.308879,0.548623,0.472101,0.172917,0.098853,0.308879,0.040193,0.182574,0.085505,0.233285,-1.080663,-0.232546,-0.406366,-0.687687,0.271886,-0.232472,0.102129,4
2,1.848564,-0.264250,-0.461423,0.409400,1.305455,2.329398,0.370965,0.090167,0.107958,0.0,0.395874,0.308879,0.548623,0.472101,0.172917,0.098853,0.308879,0.040193,0.182574,0.085505,0.233285,0.925358,1.459782,1.221876,1.877777,0.271886,-0.232472,0.102129,2
3,0.511733,-0.264250,-0.461423,0.409400,-0.525726,-0.276144,0.370965,0.090167,0.107958,0.0,0.395874,0.308879,-1.999287,-2.118189,0.172917,0.098853,-3.237512,0.040193,0.182574,0.085505,0.233285,0.925358,-0.008030,-0.406366,1.504523,0.271886,-0.232472,0.102129,2
4,-0.825098,-0.264250,-0.461423,0.409400,-0.525726,-0.276144,0.370965,0.090167,0.107958,0.0,-2.526055,0.308879,-1.999287,-2.118189,0.172917,0.098853,0.308879,0.040193,0.182574,0.085505,0.233285,0.925358,-0.573268,-1.164793,1.877777,0.271886,-0.232472,0.102129,2
5,-0.825098,-0.264250,-0.461423,0.409400,1.305455,-0.276144,-2.695676,0.090167,0.107958,0.0,0.395874,0.308879,-1.999287,0.472101,0.172917,0.098853,0.308879,0.040193,0.182574,0.085505,0.233285,-1.080663,0.443257,1.087230,-0.687687,0.271886,-0.232472,0.102129,3
6,-0.825098,-0.264250,-0.461423,0.409400,-0.525726,-0.276144,-2.695676,0.090167,0.107958,0.0,0.395874,0.308879,0.548623,0.472101,0.172917,0.098853,0.308879,0.040193,0.182574,0.085505,0.233285,0.925358,-0.233674,1.332042,1.877777,0.271886,-0.232472,0.102129,2
7,0.511733,-0.264250,-0.461423,0.409400,-0.525726,-0.276144,0.370965,0.090167,0.107958,0.0,0.395874,0.308879,-1.999287,-2.118189,0.172917,0.098853,0.308879,0.040193,0.182574,0.085505,0.233285,0.925358,-0.910605,-0.406121,-0.608830,0.271886,-0.232472,0.102129,2
8,-0.379487,4.550071,-0.461423,0.409400,1.305455,-0.276144,0.370965,0.090167,0.107958,0.0,0.395874,0.308879,-0.725332,0.472101,0.172917,0.098853,0.308879,0.040193,0.182574,0.085505,0.233285,-1.080663,1.233010,-0.332678,-0.687687,0.271886,3.727218,0.102129,2
9,0.066123,-0.264250,-0.461423,0.409400,1.305455,2.665597,0.370965,0.090167,0.107958,0.0,0.395874,-3.237512,-0.725332,0.472101,0.172917,0.098853,0.308879,0.040193,0.182574,0.085505,0.233285,-1.080663,0.443257,-0.406121,-0.687687,0.271886,-0.232472,0.102129,2


Here you can see that there are about 27 independet variables i.e from f0 to f27 and target variable grade

In [0]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 620 entries, 0 to 619
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   f0      620 non-null    float64
 1   f1      620 non-null    float64
 2   f2      620 non-null    float64
 3   f3      620 non-null    float64
 4   f4      620 non-null    float64
 5   f5      620 non-null    float64
 6   f6      620 non-null    float64
 7   f7      620 non-null    float64
 8   f8      620 non-null    float64
 9   f9      620 non-null    float64
 10  f10     620 non-null    float64
 11  f11     620 non-null    float64
 12  f12     620 non-null    float64
 13  f13     620 non-null    float64
 14  f14     620 non-null    float64
 15  f15     620 non-null    float64
 16  f16     620 non-null    float64
 17  f17     620 non-null    float64
 18  f18     620 non-null    float64
 19  f19     620 non-null    float64
 20  f20     620 non-null    float64
 21  f21     620 non-null    float64
 22  f2

In [0]:
train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
f0,620.0,-1.344802e-16,1.000807,-0.825098,-0.825098,-0.379487,0.511733,2.294174
f1,620.0,6.596874e-16,1.000807,-0.264250,-0.264250,-0.264250,-0.264250,4.920404
f2,620.0,3.697759e-17,1.000807,-0.461423,-0.461423,-0.461423,-0.461423,3.032397
f3,620.0,6.503758e-16,1.000807,-2.442599,0.409400,0.409400,0.409400,0.409400
f4,620.0,-9.454803e-17,1.000807,-2.356907,-0.525726,-0.525726,1.305455,1.305455
f5,620.0,1.146037e-16,1.000807,-0.276144,-0.276144,-0.276144,-0.276144,5.607339
f6,620.0,1.577591e-16,1.000807,-2.695676,0.370965,0.370965,0.370965,0.370965
f7,620.0,-2.734820e-16,1.000807,-11.090537,0.090167,0.090167,0.090167,0.090167
f8,620.0,6.083843e-17,1.000807,-13.278881,0.107958,0.107958,0.107958,0.107958
f9,620.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Target variable has 0 to 4 different values i.e. total 5 different values , so it means that we need to predict the class.
f0 to f27 are the variable

In [0]:
#lets whether data has some null values or not

train_df.isnull().any()

f0       False
f1       False
f2       False
f3       False
f4       False
f5       False
f6       False
f7       False
f8       False
f9       False
f10      False
f11      False
f12      False
f13      False
f14      False
f15      False
f16      False
f17      False
f18      False
f19      False
f20      False
f21      False
f22      False
f23      False
f24      False
f25      False
f26      False
f27      False
grade    False
dtype: bool

In [0]:
#lets check the target variable distribution in dataset
train_df.grade.value_counts()

2    472
1     68
3     47
4     27
0      6
Name: grade, dtype: int64

You can see that grade 0 has least no of records and grade 2 has max no of records , so we need to upscale the data

In [0]:
#lets do dimension reduction , normalization and upscaling

pca = PCA()
scale = StandardScaler()
sm = SMOTE()

x = train_df.drop(columns='grade')
y = train_df.grade
x1 , y1 = sm.fit_resample(x,y)
x_scale = scale.fit_transform(x1)
x_pca = pca.fit_transform(x_scale)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

In [0]:

#defining tunning parameter
params = {
    'learning_rate' : [0.1,0.001,0.01],
    'max_depth': [3,5,7,11],
    'min_child_weight' : [1,2,3,5],
    'lamdba':[0.5,1,2,0.2,0.3],
    'gamma':[0.1,0.2,0.3],
    'scale_pos_weight' : [1,3,2,0.5],
    'objective':'multi:softmax',
    'num_class': [4],
    'subsample':[0.5,1],
    'n_estimators':[1000],
    'early_stopping_rounds':[2],
    'colsample_bytree':[0.5,0.6,0.7,0.8,0.9 ],
    'num_boost_round' :[999]
}


#defining model
classifier = xgb.XGBClassifier()

#cross calidation fold for data validation 
stf = StratifiedKFold(n_splits=5,shuffle=True,random_state=123)

#parameter tunning algo
random = RandomizedSearchCV(estimator=classifier,param_distributions=params,n_iter=50,scoring='neg_log_loss',cv=stf,verbose=1,n_jobs=-1)

In [0]:
#fitting model

random.fit(x_pca,y1)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 10.4min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 38.8min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed: 48.1min finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=123, shuffle=True),
                   error_score=nan,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=3, min_child_weight=1,
                                           missing=None, n_estimators=100,
                                           n_jobs=1, nthread=None,
                                           objective='bin...
                                        'lamdba': [0.5, 1, 2, 0.2, 0.3],
                                        'learning_rate': [0.1, 0.001, 0.01],
                                        'max_depth': [3, 5, 7, 11],
                            

In [0]:
#save the model

from joblib import load,dump
dump(random,'MetalFurnace-Participants_Data/xgb_alloygrad_model_v2.joblib')

['MetalFurnace-Participants_Data/xgb_alloygrad_model_v2.joblib']

In [0]:
#Lets read the test file and predict the probabilities and save the result into file

#read file
test_df = pd.read_csv("MetalFurnace-Participants_Data/Test.csv")

#transforming data as look like train data
test = pca.transform(scale.transform(test_df))

#predicting the probabilities
y_predicted = random.predict_proba(test)

In [0]:
y_predicted

array([[8.2438282e-06, 5.8877034e-05, 9.9983251e-01, 2.3628301e-05,
        7.6711294e-05],
       [3.4080411e-05, 1.7166214e-05, 2.9953071e-03, 9.9685127e-01,
        1.0220550e-04],
       [1.3025860e-04, 2.4466046e-05, 9.9902856e-01, 2.0629482e-04,
        6.1044551e-04],
       ...,
       [7.4440596e-04, 9.7977640e-03, 9.8753637e-01, 1.0246943e-03,
        8.9677057e-04],
       [4.9194685e-05, 9.2813370e-06, 9.9963617e-01, 2.2498869e-04,
        8.0387690e-05],
       [2.6538188e-04, 4.6853069e-04, 9.9780506e-01, 7.8210572e-04,
        6.7889626e-04]], dtype=float32)

In [0]:
final_result = pd.DataFrame(data=y_predicted,columns=[0,1,2,3,4])

In [0]:
final_result.to_csv('MetalFurnace-Participants_Data/final_submission_xgb_V2.csv',index=False)